In [2]:
# Etapa 1: Query SQL para histórico ponto

# Caminho físico do lakehouse de destino
path_destino = "abfss://ws_departamento_pessoal@onelake.dfs.fabric.microsoft.com/lk_ponto.Lakehouse/Tables/tab_gold_fato_painel_ponto_historico_base"

pfmovcc = spark.read.format("delta").load(
  "abfss://ws_dados_central@onelake.dfs.fabric.microsoft.com/lk_AutomacaoFolhaPagamento.Lakehouse/Tables/pfmovcc"
)
pfmovcc.createOrReplaceTempView("pfmovcc")

pfmovcccompl = spark.read.format("delta").load(
  "abfss://ws_dados_central@onelake.dfs.fabric.microsoft.com/lk_AutomacaoFolhaPagamento.Lakehouse/Tables/pfmovcccompl"
)
pfmovcccompl.createOrReplaceTempView("pfmovcccompl")

pfcompl = spark.read.format("delta").load(
  "abfss://ws_dados_central@onelake.dfs.fabric.microsoft.com/lk_AutomacaoFolhaPagamento.Lakehouse/Tables/pfcompl"
)
pfcompl.createOrReplaceTempView("pfcompl")

pevento = spark.read.format("delta").load(
  "abfss://ws_dados_central@onelake.dfs.fabric.microsoft.com/lk_AutomacaoFolhaPagamento.Lakehouse/Tables/pevento"
)
pevento.createOrReplaceTempView("pevento")

gccusto = spark.read.format("delta").load(
  "abfss://ws_dados_central@onelake.dfs.fabric.microsoft.com/lk_InteligenciaNegocios.Lakehouse/Tables/gccusto"
)
gccusto.createOrReplaceTempView("gccusto")


df_resultado = spark.sql("""
WITH base AS (
    SELECT
        f.CODPESSOA,
        mv.CODCOLIGADA,
        f.CODFILIAL,
        mv.CODCCUSTO                                   AS CCUSTO,
        gc.CODCCUSTO                                   AS CODCCUSTO_GC,
        gc.NOME                                        AS NOME_GC,
        f.CODFUNCAO,
        fu.NOME                                        AS FUNCAO,
        mv.ANOCOMP,
        mv.MESCOMP,
        mv.CHAPA,
        f.DATAADMISSAO,
        f.DATADEMISSAO,
        CASE WHEN co.OBRA1 = 1341 THEN 'S' ELSE 'N' END AS QF,
        ev.DESCRICAO,
        ev.PROVDESCBASE,
        ev.TOTALIZADOR,
        ev.VALHORDIAREF,
        mv.REF,
        mv.VALOR,
        mv.CODEVENTO,

        CASE
            WHEN upper(ev.DESCRICAO) LIKE '%EXTRA%'
            THEN 'HORAS EXTRAS'
            ELSE 'HORAS NORMAIS'
        END AS TIPO,

        CASE
            WHEN mv.CODEVENTO = 0002 THEN 0
            ELSE
                CASE ev.PROVDESCBASE
                    WHEN 'P' THEN
                        CASE WHEN ev.TOTALIZADOR = '02' THEN
                            CASE ev.VALHORDIAREF
                                WHEN 'V' THEN 0
                                WHEN 'D' THEN mv.REF * 7.3333
                                ELSE mv.REF
                            END
                        ELSE 0 END
                    WHEN 'D' THEN
                        (CASE WHEN ev.TOTALIZADOR = '02' THEN
                            CASE ev.VALHORDIAREF
                                WHEN 'V' THEN 0
                                WHEN 'D' THEN mv.REF * 7.3333
                                ELSE mv.REF
                            END
                         ELSE 0 END) * -1
                    ELSE
                        CASE WHEN mv.CODEVENTO = 9001 THEN mv.REF * 7.3333 ELSE 0 END
                END
        END                                             AS part_hhs_dsr,

        CASE
            WHEN ev.PROVDESCBASE = 'P' THEN
                CASE WHEN ev.TOTALIZADOR = '02' THEN
                    CASE ev.VALHORDIAREF
                        WHEN 'V' THEN 0
                        WHEN 'D' THEN mv.REF * 7.3333
                        ELSE mv.REF
                    END
                ELSE 0 END
            ELSE
                (CASE WHEN ev.TOTALIZADOR = '02' THEN
                    CASE ev.VALHORDIAREF
                        WHEN 'V' THEN 0
                        WHEN 'D' THEN mv.REF * 7.3333
                        ELSE mv.REF
                    END
                 ELSE 0 END) * -1
        END                                             AS part_horas,

        CASE
            WHEN ev.PROVDESCBASE = 'D' THEN mv.VALOR * -1
            ELSE CASE WHEN ev.PROVDESCBASE = 'P' THEN mv.VALOR ELSE 0 END
        END                                             AS part_vlr,

        CASE
            WHEN mv.CODEVENTO = 0002 THEN 0
            ELSE
                CASE
                    WHEN ev.PROVDESCBASE = 'D' THEN mv.VALOR * -1
                    WHEN ev.PROVDESCBASE = 'P' THEN mv.VALOR
                    ELSE CASE WHEN mv.CODEVENTO = 9001 THEN mv.VALOR ELSE 0 END
                END
        END                                             AS part_vlrs_dsr
    FROM pfmovcc  AS mv
    INNER JOIN pfunc    AS f  ON mv.CODCOLIGADA = f.CODCOLIGADA AND mv.CHAPA = f.CHAPA
    INNER JOIN pfcompl  AS co ON f.CODCOLIGADA  = co.CODCOLIGADA AND f.CHAPA = co.CHAPA
    INNER JOIN pfuncao  AS fu ON f.CODCOLIGADA  = fu.CODCOLIGADA AND f.CODFUNCAO = fu.CODIGO
    INNER JOIN pevento  AS ev ON mv.CODCOLIGADA = ev.CODCOLIGADA AND mv.CODEVENTO = ev.CODIGO
    INNER JOIN gccusto  AS gc ON mv.CODCOLIGADA = gc.CODCOLIGADA AND mv.CODCCUSTO = gc.CODREDUZIDO
    WHERE
        (ev.TOTALIZADOR = 02 OR mv.CODEVENTO = 9001) AND
        mv.CODEVENTO <> 1201 AND
        COALESCE(f.TIPODEMISSAO, '') <> '5' AND
        mv.VALOR <> 0 AND
        --mv.ANOCOMP = '2023' AND
        mv.CODCOLIGADA = 2 AND
        mv.CODCCUSTO LIKE '%D'
	
	UNION	
	
SELECT
        f.CODPESSOA,
        mv.CODCOLIGADA,
        f.CODFILIAL,
        mv.CODCCUSTO                                   AS CCUSTO,
        gc.CODCCUSTO                                   AS CODCCUSTO_GC,
        gc.NOME                                        AS NOME_GC,
        f.CODFUNCAO,
        fu.NOME                                        AS FUNCAO,
        mv.ANOCOMP,
        mv.MESCOMP,
        mv.CHAPA,
        f.DATAADMISSAO,
        f.DATADEMISSAO,
        CASE WHEN co.OBRA1 = 1341 THEN 'S' ELSE 'N' END AS QF,
        ev.DESCRICAO,
        ev.PROVDESCBASE,
        ev.TOTALIZADOR,
        ev.VALHORDIAREF,
        mv.REF,
        mv.VALOR,
        mv.CODEVENTO,

        CASE
            WHEN upper(ev.DESCRICAO) LIKE '%EXTRA%'
            THEN 'HORAS EXTRAS'
            ELSE 'HORAS NORMAIS'
        END AS TIPO,
        
        CASE
            WHEN mv.CODEVENTO = 0002 THEN 0
            ELSE
                CASE ev.PROVDESCBASE
                    WHEN 'P' THEN
                        CASE WHEN ev.TOTALIZADOR = '02' THEN
                            CASE ev.VALHORDIAREF
                                WHEN 'V' THEN 0
                                WHEN 'D' THEN mv.REF * 7.3333
                                ELSE mv.REF
                            END
                        ELSE 0 END
                    WHEN 'D' THEN
                        (CASE WHEN ev.TOTALIZADOR = '02' THEN
                            CASE ev.VALHORDIAREF
                                WHEN 'V' THEN 0
                                WHEN 'D' THEN mv.REF * 7.3333
                                ELSE mv.REF
                            END
                         ELSE 0 END) * -1
                    ELSE
                        CASE WHEN mv.CODEVENTO = 9001 THEN mv.REF * 7.3333 ELSE 0 END
                END
        END                                             AS part_hhs_dsr,

        CASE
            WHEN ev.PROVDESCBASE = 'P' THEN
                CASE WHEN ev.TOTALIZADOR = '02' THEN
                    CASE ev.VALHORDIAREF
                        WHEN 'V' THEN 0
                        WHEN 'D' THEN mv.REF * 7.3333
                        ELSE mv.REF
                    END
                ELSE 0 END
            ELSE
                (CASE WHEN ev.TOTALIZADOR = '02' THEN
                    CASE ev.VALHORDIAREF
                        WHEN 'V' THEN 0
                        WHEN 'D' THEN mv.REF * 7.3333
                        ELSE mv.REF
                    END
                 ELSE 0 END) * -1
        END                                             AS part_horas,

        CASE
            WHEN ev.PROVDESCBASE = 'D' THEN mv.VALOR * -1
            ELSE CASE WHEN ev.PROVDESCBASE = 'P' THEN mv.VALOR ELSE 0 END
        END                                             AS part_vlr,

        CASE
            WHEN mv.CODEVENTO = 0002 THEN 0
            ELSE
                CASE
                    WHEN ev.PROVDESCBASE = 'D' THEN mv.VALOR * -1
                    WHEN ev.PROVDESCBASE = 'P' THEN mv.VALOR
                    ELSE CASE WHEN mv.CODEVENTO = 9001 THEN mv.VALOR ELSE 0 END
                END
        END                                             AS part_vlrs_dsr
    FROM pfmovcccompl  AS mv
    INNER JOIN pfunc    AS f  ON mv.CODCOLIGADA = f.CODCOLIGADA AND mv.CHAPA = f.CHAPA
    INNER JOIN pfcompl  AS co ON f.CODCOLIGADA  = co.CODCOLIGADA AND f.CHAPA = co.CHAPA
    INNER JOIN pfuncao  AS fu ON f.CODCOLIGADA  = fu.CODCOLIGADA AND f.CODFUNCAO = fu.CODIGO
    INNER JOIN pevento  AS ev ON mv.CODCOLIGADA = ev.CODCOLIGADA AND mv.CODEVENTO = ev.CODIGO
    INNER JOIN gccusto  AS gc ON mv.CODCOLIGADA = gc.CODCOLIGADA AND mv.CODCCUSTO = gc.CODREDUZIDO
    WHERE
        (ev.TOTALIZADOR = 02 OR mv.CODEVENTO = 9001) AND
        mv.CODEVENTO <> 1201 AND
        COALESCE(f.TIPODEMISSAO, '') <> '5' AND
        mv.VALOR <> 0 AND
        --mv.ANOCOMP = '2023' AND
        mv.CODCOLIGADA = 2 AND
        mv.CODCCUSTO LIKE '%D'	
)
SELECT
    b.CODPESSOA,
    b.CODCOLIGADA,
    b.CODFILIAL,
    b.CCUSTO,
    b.CODCCUSTO_GC          AS CODCCUSTO,
    b.NOME_GC               AS NOME,
    b.CODFUNCAO,
    b.FUNCAO,
    b.ANOCOMP,
    b.MESCOMP,
    b.CHAPA,
    b.DATAADMISSAO,
    b.DATADEMISSAO,
    b.QF,
    b.DESCRICAO,
    b.TIPO,
    SUM(b.part_hhs_dsr)     AS HHS_DSR,
    SUM(b.part_horas)       AS HORAS,
    SUM(b.part_vlr)         AS VLR,
    SUM(b.part_vlrs_dsr)    AS VLRS_DSR
FROM base AS b
LEFT JOIN tab_gold_dim_funcionario_ultima_posicao_mes AS df 
                     ON df.CODCOLIGADA = b.CODCOLIGADA 
                    AND df.CHAPA = b.CHAPA 
                    AND YEAR(df.DATA) = b.ANOCOMP AND MONTH(df.DATA) = b.MESCOMP
LEFT JOIN tab_gold_dim_funcionario_ultima_posicao AS dfu 
                     ON dfu.CODCOLIGADA = b.CODCOLIGADA 
                    AND dfu.CHAPA = b.CHAPA 
GROUP BY
    b.CODPESSOA,
    b.CODCCUSTO_GC,
    b.NOME_GC,
    b.CODCOLIGADA,
    b.CODFILIAL,
    b.CODFUNCAO,
    b.FUNCAO,
    b.ANOCOMP,
    b.MESCOMP,
    b.DESCRICAO,
    b.TIPO,
    b.CHAPA,
    b.DATAADMISSAO,
    b.DATADEMISSAO,
    b.QF,
    b.CCUSTO

""")

# Salva no Lakehouse de destino usando caminho físico
df_resultado.write.format("delta").mode("overwrite").save(path_destino)


StatementMeta(, f1c3c4d2-9afe-4078-b8d6-e54cb37b8c65, 4, Finished, Available, Finished)